In [3]:
import os
import sys
from typing import List, Tuple

In [4]:
#Functions

FactorList = List[Tuple[int, int]]

def validate_pairs(n: int, allowed_pairs: FactorList):
    """Checks for invalid pairs, duplicate pairs (up to ordering), and self-loops."""
    elements = [u for pair in allowed_pairs for u in pair]
    assert 1 <= min(elements) and max(elements) <= n
    for (u, v) in allowed_pairs:
        assert u != v, f"Self-loop found: {(u, v)}"
    pairs = set([(min(u, v), max(u, v)) for (u, v) in allowed_pairs])
    assert len(pairs) == len(allowed_pairs), "Duplicate edge found."
    

def is_transitive(n, factors):
    return Graph([list(range(1, n + 1)), factors]).is_connected()


def change_index_start(target, allowed_pairs, start_index=1):
    """Reduce all numbers in target and allowed_pairs by start_index."""
    for index, ele in enumerate(target):
        target[index] = ele - start_index
    for index, (u, v) in enumerate(allowed_pairs):
        allowed_pairs[index] = (u - start_index, v - start_index)


def eval_factors(n: int, factors: FactorList):
    p = Permutations(n).identity()
    for factor in factors:
        p = p.left_action_product(Permutation(factor))
    return p


def get_min_transitive_factorizations(
        target: Permutation,
        allowed_pairs: FactorList) -> Tuple[int, List[FactorList]]:
    """Find the list of minimal transitive factorizations of target using only the
    transpositions in allowed_pairs.
    target and allowed_pairs are 1-indexed.
    Returns:
        (min length, list of minimal transitive factorizations) (int, List[FactorList])
        Returns (0, []) instead if the given pairs do not connect [n].
    """
    n = target.size()  # permutation size

    if not is_transitive(n, allowed_pairs):
        return (0, [])

    generator = RecursivelyEnumeratedSet(
        [[]],
        lambda fact: [fact + [x] for x in allowed_pairs],
        structure='forest')
    generator_it = generator.breadth_first_search_iterator()

    min_length = -1  # -1 if not found yet
    last_length = -1
    factorizations = []
    while True:
        factors = next(generator_it)
        if min_length != -1 and len(factors) > min_length:
            break

        if len(factors) > last_length:
            last_length = len(factors)
            print(f"Trying factorization length {len(factors)}")

        if not is_transitive(n, factors) or not eval_factors(n, factors) == target:
            continue
        min_length = len(factors)
        factorizations.append(factors)

    return min_length, factorizations



def main(target, allowed_pairs):
    #target, allowed_pairs = read_file(filepath)
    target = Permutation(target)
    n = target.size()
    validate_pairs(n, allowed_pairs)

    print("-----------------------------")
    print(f"target: {target}")
    print(f"allowed_pairs: {allowed_pairs}")

    min_length, min_facts = get_min_transitive_factorizations(
        target, allowed_pairs)
    print("-----------------------------")

    if min_length == 0:
        eprint(f"Given transpositions do not connect [{len(target)}].")
    else:
        print(f"Min length: {min_length}")
        print(f"Number of minimal transitive factorizations: {len(min_facts)}")
        for factors in min_facts:
            print(' '.join([str(factor) for factor in factors]))



NameError: name 'Permutation' is not defined

In [ ]:
#Try an example:

test_target = [2, 1, 3]
test_allowed_pairs = [(1,2), (1,3)]
main(test_target, test_allowed_pairs)

-----------------------------
target: [2, 1, 3]
allowed_pairs: [(1, 2), (1, 3)]
Trying factorization length 0
Trying factorization length 1
Trying factorization length 2
Trying factorization length 3
-----------------------------
Min length: 3
Number of minimal transitive factorizations: 2
(1, 2) (1, 3) (1, 3)
(1, 3) (1, 3) (1, 2)


In [ ]:
#Try an example:

test_target = [2, 4, 1, 3]
test_allowed_pairs = [(1,2), (1,3), (1,4)]
main(test_target, test_allowed_pairs)

-----------------------------
target: [2, 4, 1, 3]
allowed_pairs: [(1, 2), (1, 3), (1, 4)]
Trying factorization length 0
Trying factorization length 1
Trying factorization length 2
Trying factorization length 3
-----------------------------
Min length: 3
Number of minimal transitive factorizations: 1
(1, 3) (1, 4) (1, 2)
